# Projekt Preference learning
### Maciej Wieczorek, 148141
### Kacper Perz, 145261

## 1. Zbiór danych

In [ ]:
import pandas as pd

dataset = pd.read_csv('cpu.csv', header=None, names=['f1','f2','f3','f4','f5','f6','class'])
dataset.loc[dataset['class'] < 2, 'class'] = int(0)
dataset.loc[dataset['class'] >= 2, 'class'] = int(1)
X = dataset.loc[:, dataset.columns != 'class']
y = dataset['class']
dataset.head()


## 2. Prosty, interpretowalny model ML

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.preprocessing import label_binarize
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt


# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

# Initialize the model
model = xgb.XGBClassifier(
    objective='binary:logistic',  # for multi-class classification
    max_depth=4,
    learning_rate=0.05,
    n_estimators=100,
    random_state=42
)

# Train the model
model.fit(X_train.values, y_train)

# Make predictions
pred_probs = model.predict_proba(X_test.values)  # This returns a probability distribution over classes
preds = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, preds)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Binarize the true labels for AUC calculation
y_test_bin = label_binarize(y_test, classes=[0, 1])

roc = roc_auc_score(preds, y_test)
print(f"AUC: {roc:.2f}")

# Calculate F1-score
f1 = f1_score(y_test, preds, average='weighted')
print(f"F1-score: {f1:.2f}")

In [ ]:
dump_list = model.get_booster().get_dump()
num_trees = len(dump_list)
print(num_trees)

In [ ]:
import matplotlib.pyplot as plt

xgb.plot_tree(model, num_trees=1)
plt.show()

In [ ]:
feature_important = model.get_booster().get_score(importance_type='weight')

In [ ]:
feature_important

In [ ]:
xgb.plot_importance(model, max_num_features = 15)

#### 2.1 Wyjaśnienie decyzji

In [ ]:
!pip install shap

import shap

In [ ]:
preds

# Zadanie 2.1.1 i 2.1.2

Dla xgboosta wzięliśmy 3 pierwsze przykłady ze zbioru testowego. Za pomocą
shap'a sprawdziliśmy, które cechy miały wpływ na predykcję danej klasy. Przedstawiają to poniższe wykresy.

Dla x_0 (klasa '0') największą siłę miała cecha f3, dla x_1 (klasa '1') cecha f4 i dla x_2 (klasa '0') cecha f3. We wszystkich przykładach zdecydowaliśmy się zmienić nieznacznie cechę f3 widząc (na wykresach na samym dole sekcji), że ma największy wpływ na największą liczbę przykładów i stanowi uniwersalny wyznacznik.

# Zadanie 2.1.3




In [ ]:
# creating an explainer for our model
explainer = shap.TreeExplainer(model)

# finding out the shap values using the explainer
shap_values = explainer.shap_values(X_test)
print(shap_values.shape)

# Expected/Base/Reference value = the value that would be predicted if we didn’t know any features of the current output”
print('Expected Value:', explainer.expected_value)

# displaying the first 5 rows of the shap values table
pd.DataFrame(shap_values).head()

In [ ]:
x_0, y_0 = X_test.iloc[0,:], y_test.values[0]
x_1, y_1 = X_test.iloc[1,:], y_test.values[1]
x_2, y_2 = X_test.iloc[2,:], y_test.values[2]

print(x_0, y_0)
print(x_1, y_1)
print(x_2, y_2)

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value,
                shap_values[0,:], x_0) # ma label = 0

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value,
                shap_values[1,:], x_1) # ma label = 1

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value,
                shap_values[2,:], x_2) # ma label = 0

In [ ]:
# f4 u przykladow o klasie = 0 wynosi czesto 0. shap podpowiada, ze to wlasnie ta cecha
# i ta wartosc cechy popycha przykladow od klasy 0
# zwiekszmy te ceche i zrobmy predykcje

x_0['f3'] = 1.2
print(x_0.values.reshape(1, 6).shape)

print(model.predict(x_0.values.reshape(1,6)))

In [ ]:
# przyklad x_1 nalezy do klasy '1'. zmienmy 'f4' na 0

x_1['f3'] = 0.08
print(x_1.values.reshape(1, 6).shape)

print(model.predict(x_1.values.reshape(1,6)))

In [ ]:
# przyklad x_2 nalezy do klasy '0'. zmienmy 'f4' na 0.1

x_2['f3'] = 0.12
print(x_2.values.reshape(1, 6).shape)

print(model.predict(x_2.values.reshape(1,6)))

In [ ]:
shap.initjs()
shap.summary_plot(shap_values,
                  X_test, plot_type="bar")

In [ ]:
shap.initjs()
shap.summary_plot(shap_values, X_test)

# wniosek: f3 ma wpływ na wszystkie przykłady i z największą siłą, jako że jest na szczycie
# i jako że wartości SHAP są dla tej cechy największe

## Próbkowanie przestrzeni
Hipoteza: wieksza wartosc f3 powinna spowodowac wiecej predykcji klasy '1'

Po zmianie f3: hipoteza potwierdzona.

In [ ]:
# hipoteza: wieksza wartosc f3 powinna spowodowac wiecej predykcji klasy '1'

import numpy as np


X_hip_test = X_test.copy()
X_hip_pred = model.predict(X_hip_test)
print('Licznosc klasy 0:', np.sum(y_test == 0))
print('Licznosc klasy 1:', np.sum(y_test == 1))
print('Licznosc predykcji klasy 0:', np.sum(X_hip_pred == 0))
print('Licznosc predykcji klasy 1:', np.sum(X_hip_pred == 1))

X_hip_test['f3'] += 0.04
X_hip_pred = model.predict(X_hip_test)
print('Po zmianie f3. Licznosc predykcji klasy 0:', np.sum(X_hip_pred == 0))
print('Po zmianie f3. Licznosc predykcji klasy 1:', np.sum(X_hip_pred == 1))

# --------
# X_hip_test = X_test.copy()
# X_hip_pred = model.predict(X_hip_test)
# print('Licznosc klasy 0:', np.sum(y_test == 0))
# print('Licznosc klasy 1:', np.sum(y_test == 1))
# print('Licznosc predykcji klasy 0:', np.sum(X_hip_pred == 0))
# print('Licznosc predykcji klasy 1:', np.sum(X_hip_pred == 1))

# X_hip_test['f1'] -= 1
# X_hip_pred = model.predict(X_hip_test)
# print('Po zmianie f1. Licznosc predykcji klasy 0:', np.sum(X_hip_pred == 0))
# print('Po zmianie f1. Licznosc predykcji klasy 1:', np.sum(X_hip_pred == 1))


# --------
# X_hip_test = X_test.copy()
# X_hip_pred = model.predict(X_hip_test)
# print('Licznosc klasy 0:', np.sum(y_test == 0))
# print('Licznosc klasy 1:', np.sum(y_test == 1))
# print('Licznosc predykcji klasy 0:', np.sum(X_hip_pred == 0))
# print('Licznosc predykcji klasy 1:', np.sum(X_hip_pred == 1))

# X_hip_test['f2'] += 1
# X_hip_pred = model.predict(X_hip_test)
# print('Po zmianie f2. Licznosc predykcji klasy 0:', np.sum(X_hip_pred == 0))
# print('Po zmianie f2. Licznosc predykcji klasy 1:', np.sum(X_hip_pred == 1))

# --------
# X_hip_test = X_test.copy()
# X_hip_pred = model.predict(X_hip_test)
# print('Licznosc klasy 0:', np.sum(y_test == 0))
# print('Licznosc klasy 1:', np.sum(y_test == 1))
# print('Licznosc predykcji klasy 0:', np.sum(X_hip_pred == 0))
# print('Licznosc predykcji klasy 1:', np.sum(X_hip_pred == 1))

# X_hip_test['f4'] += 0.1
# X_hip_pred = model.predict(X_hip_test)
# print('Po zmianie f4. Licznosc predykcji klasy 0:', np.sum(X_hip_pred == 0))
# print('Po zmianie f4. Licznosc predykcji klasy 1:', np.sum(X_hip_pred == 1))


# -----------------------
# X_hip_test = X_test.copy()
# X_hip_pred = model.predict(X_hip_test)
# print('Licznosc klasy 0:', np.sum(y_test == 0))
# print('Licznosc klasy 1:', np.sum(y_test == 1))
# print('Licznosc predykcji klasy 0:', np.sum(X_hip_pred == 0))
# print('Licznosc predykcji klasy 1:', np.sum(X_hip_pred == 1))

# X_hip_test['f6'] -= 20
# X_hip_pred = model.predict(X_hip_test)
# print('Po zmianie f5. Licznosc predykcji klasy 0:', np.sum(X_hip_pred == 0))
# print('Po zmianie f5. Licznosc predykcji klasy 1:', np.sum(X_hip_pred == 1))






# rosnace - f2, f4 (powyzej 0.1 rzeczy juz sa nierozroznialne i zawsze wedruja do klasy 1)
# malejace - f1


#### 2.2 Interpretacja modelu

Czy na podstawie uzyskanych parametrów możemy powiedzieć coś o preferencjach użytkowników?

Zazwyczaj tak. Jeżeli dana cecha ma znaczący wpływ na przydział do danej klasy, to znaczy własnie, że jakaś cecha odgrywa większą rolę w klasyfikacji danego przykładu. Np. wysoka wartość na f3 może mieć decydujący wpływ na przydział do klasy '1' i przyrost jednostki w na tej cesze będzie preferowany nad proporcjonalny przyrost na innej cesze.

In [ ]:
shap.initjs()
shap.summary_plot(shap_values, X_test, plot_type="bar")

Jaki jest wpływ każdego z kryteriów? Czy są jakieś kryteria, które nie mają żadnego znaczenia, czy też mają wpływ decydujący.

Oprócz powyższego wykresu (ważność cech według xgboosta) szerszej odpowiedzi udziela także wykres poniższy, generowany przez shap'a. Z ilustracji można wyciągnąć następujące wnioski:
- Cechy są sortowane według sumy wartości SHAP we wszystkich próbkach.
- Warto zauważyć, że f3 ma większy wpływ na model niż f4. (Ponieważ znajdują się one na górze, a pasek Feature Value dla obu cech wskazuje tam po mniej więcej po równo)
- Dla najmniejszej liczby przykładów znaczenie mają cechy f5 i f6, chociaż jeżeli już dla jakichś mają, to mają ten wpływ całkiem spory (czerwony kolor na feature value), w przeciwieństwie do f1, które może być istotne dla odrobinę większej liczby przykładów, ale bez większej wartości (brak czerwonego).


In [ ]:
shap.initjs()
shap.summary_plot(shap_values, X_test)

Jaki jest charakter danego kryterium: zysk, koszt, niemonotoniczne?


Kryterium f2 i f4 jest typu zysk, przy czym powyżej pewnej wartości zwracana klasa zawsze wynosi '1'.
Kryterium f1 jest typu koszt.

Do wyciągnięcia tych wniosków zwiększaliśmy i zmniejszaliśmy wartości kolejnych cech przykładów testowych, a następnie tak zmienione przykłady przepuszczaliśmy przez wcześniej wytrenowany model i patrzyliśmy czy i w jakich proporcjach zmieniły się predykcje.

Czy istnieją jakieś progi preferencji? Czy istnieją oceny kryteriów, które są nierozróżnialne z punktu widzenia preferencji?

Dobrym przykładem jest f4. Cecha f4 dla przykładów o klasie '0' ma wartość=0. Jeżeli zwiększymy tę wartość

## 3. interpretowalny model ANN-MCDA

In [ ]:
import pandas as pd
import numpy as np
import shap
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

dataset = pd.read_csv('cpu.csv', header=None, names=['f1','f2','f3','f4','f5','f6','class'])
dataset.loc[dataset['class'] < 2, 'class'] = 0
dataset.loc[dataset['class'] >= 2, 'class'] = 1
X = dataset.loc[:, dataset.columns != 'class'].to_numpy()
y = dataset['class'].to_numpy()
dataset.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

In [ ]:
import tensorflow as tf

class ANNChConstr(tf.keras.Model):
    def __init__(self, num_features):
        super(ANNChConstr, self).__init__()
        self.num_features = num_features
        
        # wagi kryteriów
        self.wj = tf.Variable(tf.ones([num_features]), constraint=tf.keras.constraints.NonNeg())
        # wagi interakcji
        self.wjl = tf.Variable(tf.zeros([num_features, num_features]))
    
    def call(self, inputs):
        # Obliczanie sumy ważonej dla każdego kryterium
        weighted_sum = tf.linalg.matvec(inputs, self.wj)
        
        # Obliczanie interakcji między kryteriami
        interactions = 0
        for i in range(self.num_features):
            for j in range(i + 1, self.num_features):
                interactions += self.wjl[i, j] * tf.math.minimum(inputs[:, i], inputs[:, j])
        
        # Łączenie wyników z obu warstw
        output = weighted_sum + interactions
        
        # Normalizacja
        normalization_factor = tf.reduce_sum(self.wj) + tf.reduce_sum(self.wjl)
        normalized_output = output / normalization_factor
        
        return normalized_output

def mobius_transform(X):
    # ???
    return X

def avg_regret_loss(y_true, y_pred):
    regret = tf.maximum(y_true - y_pred, y_pred - y_true, 0)
    return tf.reduce_mean(regret)

mobius_transform(X_train)
model = ANNChConstr(X_train.shape[1])
model.compile(optimizer='adam', loss=avg_regret_loss)
model.fit(X_train, y_train, epochs=1000, batch_size=X_train.shape[0], verbose=0)
model.evaluate(X_test, y_test, batch_size=X_test.shape[0])

In [ ]:
def get_prediction_classes(y_pred):
    return (y_pred > 0.5).astype(np.uint8)

y_pred = get_prediction_classes(model.predict(X_test))
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}')

In [ ]:
model.summary()

In [ ]:
wj = model.get_weights()[0]
wjl = model.get_weights()[1]
print('Wagi kryteriów')
print(np.round(wj, 4))
print('Wagi interakcji (tylko górna przekątna)')
print(np.round(wjl, 4))

#### 3.1 Wyjaśnienie decyzji

In [ ]:
print("Predykcje")
print(get_prediction_classes(model.predict(X_test[:3], verbose=0)))
print("Warianty")
print(X_test[:3])

Na przydział do klasy głównie wpływają kryteria: f3, f4 co można wywnioskować po ich wagach.
Wariant pierwszy oraz trzeci została zaklasyfikowana jako 0, ponieważ mają niskie wartości na kryteriach f3 i f4.

In [ ]:
explainer = shap.Explainer(model, X_test[:3])
shap_values = explainer(X_test[:3])
shap.plots.waterfall(shap_values[0])
shap.plots.waterfall(shap_values[1])
shap.plots.waterfall(shap_values[2])

Dla wariantu pierwszego istotne są kryteria f4, f5, i f1
Dla wariantu drugiego i trzeciego istotne są kryteria f4 oraz f3, które również mają wysoką wagę interakcji

#### 3.2 Interpretacja modelu

In [ ]:
def shapley_index():
    wj = model.get_weights()[0]
    wjl = model.get_weights()[1]
    wjl_sums = {}
    for i in range(len(wj)):
        for j in range(i+1, len(wj)):
            w = wjl[i][j]
            if i not in wjl_sums:
                wjl_sums[i] = 0
            if j not in wjl_sums:
                wjl_sums[j] = 0
            wjl_sums[i] += w / 2
            wjl_sums[j] += w / 2
        wjl_sums[i] += w

    return wjl_sums
shapley_index()

## 4. Złożony model sici neuronowej zawierającej kilka warstw ukrytych i nieliniową funkcję aktywacji

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(10, 5), activation='relu', max_iter=500, random_state=1)
mlp.fit(X_train, y_train)

In [ ]:
y_pred = mlp.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}, F1: {f1}, AUC: {auc}')

In [ ]:
for i, layer in enumerate(mlp.coefs_):
    print(f'layer {i}: {np.round(layer, 4)}')

#### 4.1 Wyjaśnienie decyzji

In [ ]:
# explainer = shap.KernelExplainer(mlp.predict_proba, X_train, link="logit")
# shap_values = explainer.shap_values(X_test, nsamples=100)

#### 4.2 Interpretacja modelu